# Utility functions

In [ ]:
def stratified_spatial(commune_ids,map_distances,reg2com_map,threshold):
    # randomly choose a region
    rnd_reg = np.random.permutation(14)[0] + 1
    coms = reg2com_map[str(rnd_reg).zfill(2)]
    # randomly choose a commune
    rnd_com = np.random.permutation(len(coms))[0]
    # 1 km = 0.00895305 degrees
    f = 50
    #f = 0
    training_inds = []
    testing_inds = []
    # starting from commune, add all other communes that are within rad to training and rest 
    #to test
    while len(training_inds) < threshold:
        f = f + 50
        training_inds = []
        testing_inds = []
        training_inds.append(rnd_com)
        rad = f * 0.00895305
        ind = 0
        for c in commune_ids:
            if c != coms[rnd_com]:
                d = map_distances[(coms[rnd_com],c)]
                if d < rad:
                    #put in training
                    training_inds.append(ind)
                else :
                    testing_inds.append(ind)
            ind = ind + 1

    return training_inds, testing_inds, rnd_reg, coms[rnd_com]


In [ ]:
def evaluate(XTrain,YTrain,XTest,alpha,classifier,args):
    supported_methods = ['MTL','Elastic','Ridge','Lasso','GPR','LinearRegression', 'Tree','Mixed','MultiViewGPR','MultiViewGPRSpatial','Kriging']
    if classifier not in supported_methods:
        print 'Only following classifiers are supported'
        print supported_methods
        raise ValueError('Undefined classifier specified')
        
    Vars = {}
    # learn model
    if classifier == 'MTL':
        lmodel = MultiTaskLasso(alpha=alpha,max_iter=10000)
        lmodel.fit(XTrain,YTrain)
        YPred = lmodel.predict(XTest)
    elif classifier == 'Elastic':
        lmodel = ElasticNet(alpha = alpha)
        lmodel.fit(XTrain,YTrain)
        YPred = lmodel.predict(XTest)
    elif classifier == 'Lasso':
        lmodel = Lasso(alpha = alpha)
        lmodel.fit(XTrain,YTrain)
        YPred = lmodel.predict(XTest)
    elif classifier == 'LinearRegression':
        lmodel = LinearRegression()
        lmodel.fit(XTrain,YTrain)
        YPred = lmodel.predict(XTest)
    elif classifier == 'Ridge':
        lmodel = Ridge(alpha= alpha)
        lmodel.fit(XTrain,YTrain)
        YPred = lmodel.predict(XTest)
    elif classifier == 'GPR':
        eng = args['eng']
        YPred = run_GP(XTrain,YTrain,XTest,eng)
    elif classifier == 'Tree':
        lmodel = RandomForestRegressor()
        lmodel.fit(XTrain,YTrain)
        YPred = lmodel.predict(XTest)
    elif classifier == 'Mixed':
        eng = args['eng']
        STrain = args['STrain']
        STest = args['STest']
        # first fit a elastic net
        lmodel = ElasticNet(alpha = alpha)
        lmodel.fit(XTrain,YTrain)
        # learn GP covariance function hyperparameters using lmodel as the mean function
        covhyp = train_spatial_GP(XTrain,YTrain,STrain,lmodel,eng)
        # run GPR on test data using the learnt hyperparameters
        YPred,var = test_spatial_GP(XTrain,YTrain,STrain,XTest,STest,lmodel,covhyp,eng)
        Vars['YVar'] = var
    elif classifier == 'MultiViewGPR' or classifier == 'MultiViewGPRSpatial' or classifier == 'Kriging':
        lmbd = alpha
        eng = args['eng']
        view2index = args['view2index']
        if classifier == 'MultiViewGPRSpatial' or classifier == 'Kriging':
            STrain = args['STrain']
            STest = args['STest']

        # process view 1
        XTrain1 = XTrain[:,0:view2index]
        XTest1 = XTest[:,0:view2index]
        lmodel = ElasticNet(alpha = alpha)
        lmodel.fit(XTrain1,YTrain)
        if classifier == 'MultiViewGPR':
            covhyp = train_GP(XTrain1,YTrain,lmodel,eng)
            YPred1,Var1 = test_GP(XTrain1,YTrain,XTest1,lmodel,covhyp,eng)
        elif classifier == 'Kriging':
            covhyp = train_Kriging(XTrain1, YTrain, STrain, lmodel, eng)
            YPred1,Var1 = test_Kriging(YTrain,STrain,XTest1,STest,lmodel,covhyp,eng)
        else:
            covhyp = train_spatial_GP(XTrain1,YTrain,STrain,lmodel,eng)
            YPred1,Var1 = test_spatial_GP(XTrain1,YTrain,STrain,XTest1,STest,lmodel,covhyp,eng)
        # process view 2
        XTrain2 = XTrain[:,view2index:]
        XTest2 = XTest[:,view2index:]
        lmodel = ElasticNet(alpha = alpha)
        lmodel.fit(XTrain2,YTrain)
        if classifier == 'MultiViewGPR':
            covhyp = train_GP(XTrain2,YTrain,lmodel,eng)
            YPred2,Var2 = test_GP(XTrain2,YTrain,XTest2,lmodel,covhyp,eng)
        elif classifier == 'Kriging':
            covhyp = train_Kriging(XTrain2, YTrain, STrain, lmodel, eng)
            YPred2,Var2 = test_Kriging(YTrain,STrain,XTest2,STest,lmodel,covhyp,eng)
        else:
            covhyp = train_spatial_GP(XTrain2,YTrain,STrain,lmodel,eng)
            YPred2,Var2 = test_spatial_GP(XTrain2,YTrain,STrain,XTest2,STest,lmodel,covhyp,eng)
        # combine predictions
        YPred = np.zeros([XTest.shape[0],YTrain.shape[1]])
        YVar = np.zeros([XTest.shape[0],YTrain.shape[1]])
        
        for oindex in range(YTrain.shape[1]):
            i_Var1 = 1/(Var1[:,oindex])
            i_Var2 = 1/(Var2[:,oindex])
            i_sum = i_Var1 + i_Var2
            w1 = i_Var1/i_sum
            w2 = i_Var2/i_sum

            # combined prediction
            YPred[:,oindex] = YPred1[:,oindex]*w1 + YPred2[:,oindex]*w2
            # variance of the combined prediction
            YVar[:,oindex] = w1*Var1[:,oindex] + w2*Var2[:,oindex] + w1*w2*(YPred1[:,oindex] - YPred2[:,oindex])**2
            #YVar.append([Var1[:,oindex],Var2[:,oindex]])
            Vars['YVar'] = YVar
            Vars['Var1'] = Var1
            Vars['Var2'] = Var2
    else:
        raise ValueError('Undefined model specified')
    return YPred,Vars


In [ ]:
# standard crossvalidation
def crossvalidate_standard(x,y,alpha,niters,classifier,args):
        
    rmsevec = []
    pcorrvec = []
    scorrvec = []
    ppvalvec = []
    spvalvec = []
    
    map_YPreds = []
    map_YVar1 = []
    map_YVar2 = []
    map_YVar_final = []
    
    
    ny = y.shape[1]
    ny1 = ny
    if len(args['mpi']) != 0:
        ny1 = ny + 1
    
    for ii in range(niters):
        if ii % 4 == 0 and ii > 0:
            print "Validation run %d of %d"%(ii,niters)
        kf = cross_validation.KFold(n=x.shape[0], n_folds=10, shuffle=False,
                                       random_state=1)
        for train_index, test_index in kf:
            # split data for crossvalidation
            XTrain, XTest, STrain, STest = x[train_index,:], x[test_index,:], s[train_index,:], s[test_index,:]
            YTrain, YTest = y[train_index,:], y[test_index,:]
            if classifier == 'MultiViewGPRSpatial' or classifier == 'Mixed' or classifier == 'Kriging':
                args['STrain'] = STrain
                args['STest'] = STest
                
            YPred,Vars = evaluate(XTrain,YTrain,XTest,alpha,classifier,args)
            if len(args['mpi']) != 0:
                #compute MPI
                YPredMPI = (YPred[:,0]*YPred[:,1])*0.0001
                YPredMPI = YPredMPI[:,np.newaxis]
                #add this to the YPred
                YPred = np.hstack([YPred,YPredMPI])
            map_YPreds.append((test_index,YPred))
            map_YVars = {}
            if 'YVar' in Vars.keys():
                YVar = Vars['YVar']
                map_YVar_final.append((test_index,YVar))
            if 'Var1' in Vars.keys():
                YVar1 = Vars['Var1']
                map_YVar1.append((test_index,YVar1))
            if 'Var2' in Vars.keys():
                YVar2 = Vars['Var2']
                map_YVar2.append((test_index,YVar2))
            rmse = np.zeros([ny1,])
            rmse[0:ny] = np.sqrt(np.mean((YPred[:,0:ny] - YTest)**2,axis=0))
            pcorr = np.zeros([ny1,])
            scorr = np.zeros([ny1,])
            ppval = np.zeros([ny1,])
            spval = np.zeros([ny1,])
            
            for i in range(ny):
                pcorr[i] = pearsonr(YPred[:,i],YTest[:,i])[0]
                ppval[i] = pearsonr(YPred[:,i],YTest[:,i])[1]
                scorr[i] = spearmanr(YPred[:,i],YTest[:,i])[0]
                spval[i] = spearmanr(YPred[:,i],YTest[:,i])[1]
            if len(args['mpi']) != 0:
                testmpi = args['mpi'][test_index]
                testmpi = testmpi[:,np.newaxis]
                rmse[ny1-1] = np.sqrt(np.mean((YPredMPI - testmpi)**2,axis=0))
                pcorr[ny1 - 1] = pearsonr(YPredMPI,testmpi)[0]
                ppval[ny1 - 1] = pearsonr(YPredMPI,testmpi)[1]
                scorr[ny1 - 1] = spearmanr(YPredMPI,testmpi)[0]
                spval[ny1 - 1] = spearmanr(YPredMPI,testmpi)[1]
                
            pcorrvec.append(pcorr)
            ppvalvec.append(ppval)
            scorrvec.append(scorr)
            spvalvec.append(spval)
            rmsevec.append(rmse)
            
    rmsevec = np.array(rmsevec)
    pcorrvec = np.array(pcorrvec)
    ppvalvec = np.array(ppvalvec)
    scorrvec = np.array(scorrvec)
    spvalvec = np.array(spvalvec)

    return pcorrvec,ppvalvec,scorrvec,spvalvec,rmsevec,map_YPreds,map_YVar1,map_YVar2,map_YVar_final


In [1]:
def crossvalidate_spatial(x,y,alpha,niters,classifier,args,common_com,map_distances,reg2com_map):
    
    selected_coms = []
    selected_regs = []
    #map to store the actual predictions
    map_YPreds = []
    map_YVar1 = []
    map_YVar2 = []
    map_YVar_final = []
    
    rmsevec = []
    pcorrvec = []
    scorrvec = []
    ppvalvec = []
    spvalvec = []

    ny = y.shape[1]
    ny1 = ny
    if len(args['mpi']) != 0:
        ny1 = ny + 1
    
    ii = 0
    while ii < niters:
        # split data for crossvalidation
        train_index,test_index,rnd_reg,rnd_com = stratified_spatial(common_com,map_distances,reg2com_map,225)
        if ii % 4 == 0 and ii > 0:
            print "Validation run %d of %d"%(ii,niters)
        if(True):
            ii = ii + 1
            XTrain, XTest, STrain, STest = x[train_index,:], x[test_index,:], s[train_index,:], s[test_index,:]
            YTrain, YTest = y[train_index,:], y[test_index,:]
            if classifier == 'MultiViewGPRSpatial' or classifier == 'Mixed' or classifier == 'Kriging':
                args['STrain'] = STrain
                args['STest'] = STest
            
            try:
                #YPred = evaluate(XTrain,YTrain,XTest,alpha,classifier,args)
                YPred,Vars = evaluate(XTrain,YTrain,XTest,alpha,classifier,args)
                if len(args['mpi']) != 0:
                    #compute MPI
                    YPredMPI = (YPred[:,0]*YPred[:,1])*0.0001
                    YPredMPI = YPredMPI[:,np.newaxis]
                    #add this to the YPred
                    YPred = np.hstack([YPred,YPredMPI])
                map_YPreds.append((test_index,YPred))
                map_YVars = {}
                if 'YVar' in Vars.keys():
                    YVar = Vars['YVar']
                    map_YVar_final.append((test_index,YVar))
                if 'Var1' in Vars.keys():
                    YVar1 = Vars['Var1']
                    map_YVar1.append((test_index,YVar1))
                if 'Var2' in Vars.keys():
                    YVar2 = Vars['Var2']
                    map_YVar2.append((test_index,YVar2))
                selected_coms.append(rnd_com)
                selected_regs.append(rnd_reg)
            except MatlabExecutionError:
                print "error identified"
                continue

            rmse = np.zeros([ny1,])
            rmse[0:ny] = np.sqrt(np.mean((YPred[:,0:ny] - YTest)**2,axis=0))
            pcorr = np.zeros([ny1,])
            scorr = np.zeros([ny1,])
            ppval = np.zeros([ny1,])
            spval = np.zeros([ny1,])
            
            for i in range(ny):
                pcorr[i] = pearsonr(YPred[:,i],YTest[:,i])[0]
                ppval[i] = pearsonr(YPred[:,i],YTest[:,i])[1]
                scorr[i] = spearmanr(YPred[:,i],YTest[:,i])[0]
                spval[i] = spearmanr(YPred[:,i],YTest[:,i])[1]
            if len(args['mpi']) != 0:
                testmpi = args['mpi'][test_index]
                testmpi = testmpi[:,np.newaxis]
                rmse[ny1-1] = np.sqrt(np.mean((YPredMPI - testmpi)**2,axis=0))
                pcorr[ny1 - 1] = pearsonr(YPredMPI,testmpi)[0]
                ppval[ny1 - 1] = pearsonr(YPredMPI,testmpi)[1]
                scorr[ny1 - 1] = spearmanr(YPredMPI,testmpi)[0]
                spval[ny1 - 1] = spearmanr(YPredMPI,testmpi)[1]
            pcorrvec.append(pcorr)
            ppvalvec.append(ppval)
            scorrvec.append(scorr)
            spvalvec.append(spval)
            rmsevec.append(rmse)

    rmsevec = np.array(rmsevec)
    pcorrvec = np.array(pcorrvec)
    ppvalvec = np.array(ppvalvec)
    scorrvec = np.array(scorrvec)
    spvalvec = np.array(spvalvec)

    return pcorrvec,ppvalvec,scorrvec,spvalvec,rmsevec,map_YPreds,map_YVar1,map_YVar2,map_YVar_final

In [ ]:
#run GP MATLAB model
def run_GP(xtrain,ytrain,xtest,eng):
    ystar = np.zeros([xtest.shape[0],ytrain.shape[1]])
    for oindex in range(ytrain.shape[1]):
        _ystar = eng.runGP(np.transpose(xtrain).tolist(),
                           np.transpose(ytrain[:,oindex:oindex+1]).tolist(),np.transpose(xtest).tolist(),
                           xtrain.shape[0],xtest.shape[0],xtrain.shape[1])
      
        _ystar = np.array(_ystar)
        ystar[:,oindex] = _ystar.flatten()
    return ystar

In [ ]:
#learn hyperparameters for GP (assuming 0 mean and SE covariance function)
def train_Kriging(xtrain,ytrain,strain,linearmodel,eng):
    y_hat_train = linearmodel.predict(xtrain)
    noise_train = ytrain - y_hat_train
    # assume ell, sf, sn
    covhyp = np.zeros([3,ytrain.shape[1]])
    means = np.transpose(linearmodel.coef_)
    for oindex in range(ytrain.shape[1]):

        ### need to pass the strain as xtrain
        _covhyp = eng.trainGP_cov(np.transpose(strain).tolist(),
                           np.transpose(noise_train[:,oindex:oindex+1]).tolist(),
                           strain.shape[0],strain.shape[1])
        _covhyp = np.array(_covhyp).flatten()
        covhyp[:,oindex] = _covhyp.flatten()
    
    return covhyp

In [ ]:
#predict using GPR
def test_Kriging(ytrain,strain,xtest,stest,linearmodel,covhyp,eng):
    ystar_test = np.zeros([stest.shape[0],ytrain.shape[1]])
    ystar_var = np.zeros([stest.shape[0],ytrain.shape[1]])
    
    yhat_test = linearmodel.predict(xtest)
    for oindex in range(ytrain.shape[1]):
        
        ### need to pass the latlon for train and test data

        _ystar_res = eng.testGP_ZeroMean_cov(np.transpose(strain).tolist(),
                                np.transpose(ytrain[:,oindex:oindex+1]).tolist(),
                                np.transpose(stest).tolist(),
                                strain.shape[0],stest.shape[0],strain.shape[1],
                                np.transpose(covhyp[:,oindex:oindex+1]).tolist())
        
        _ystar_res = np.array(_ystar_res).flatten()
        _ystar_var = _ystar_res[xtest.shape[0]:]
        _ystar_test = _ystar_res[0:xtest.shape[0]]
        
        ystar_test[:,oindex] = _ystar_test.flatten()
        ystar_var[:,oindex] = _ystar_var.flatten()
    ystar_test = ystar_test + yhat_test    
    return ystar_test, ystar_var 

In [ ]:
#learn hyperparameters for GP (assuming 0 mean and SE covariance function)
def train_GP(xtrain,ytrain,linearmodel,eng):
    y_hat_train = linearmodel.predict(xtrain)
    noise_train = ytrain - y_hat_train
    # assume ell, sf, sn
    covhyp = np.zeros([3,ytrain.shape[1]])
    means = np.transpose(linearmodel.coef_)
    for oindex in range(ytrain.shape[1]):
        #check for nzcoefs
        nzcoefs = np.where(means[:,oindex] != 0)[0]
        xtrain1 = xtrain[:,nzcoefs]
        ### need to pass the latlon for train data
        _covhyp = eng.trainGP_cov(np.transpose(xtrain1).tolist(),
                           np.transpose(noise_train[:,oindex:oindex+1]).tolist(),
                           xtrain1.shape[0],xtrain1.shape[1])
        _covhyp = np.array(_covhyp).flatten()
        covhyp[:,oindex] = _covhyp.flatten()
    
    return covhyp

In [ ]:
#predict using GPR
def test_GP(xtrain,ytrain,xtest,linearmodel,covhyp,eng):
    ystar_test = np.zeros([xtest.shape[0],ytrain.shape[1]])
    ystar_var = np.zeros([xtest.shape[0],ytrain.shape[1]])
    
    means = np.transpose(linearmodel.coef_)
    consts = linearmodel.intercept_
    
    for oindex in range(ytrain.shape[1]):
        #check for nzcoefs
        nzcoefs = np.where(means[:,oindex] != 0)[0]
        
        meanparams = np.zeros([len(nzcoefs)+1,])
        meanparams[0:len(nzcoefs)] = means[nzcoefs,oindex]
        meanparams[len(nzcoefs)] = consts[oindex]
        
        xtrain1 = xtrain[:,nzcoefs]
        xtest1 = xtest[:,nzcoefs]
        ### need to pass the latlon for train and test data

        _ystar_res = eng.testGP_cov(np.transpose(xtrain1).tolist(),
                                np.transpose(ytrain[:,oindex:oindex+1]).tolist(),
                                np.transpose(xtest1).tolist(),
                                xtrain.shape[0],xtest1.shape[0],xtrain1.shape[1],
                                np.transpose(meanparams).tolist(),
                                np.transpose(covhyp[:,oindex:oindex+1]).tolist())
        
        _ystar_res = np.array(_ystar_res).flatten()
        _ystar_var = _ystar_res[xtest1.shape[0]:]
        _ystar_test = _ystar_res[0:xtest1.shape[0]]
        
        ystar_test[:,oindex] = _ystar_test.flatten()
        ystar_var[:,oindex] = _ystar_var.flatten()
        
    return ystar_test, ystar_var 

In [ ]:
#learn hyperparameters for GP (assuming 0 mean and SE covariance function)
def train_spatial_GP(xtrain,ytrain,strain,linearmodel,eng):
    y_hat_train = linearmodel.predict(xtrain)
    noise_train = ytrain - y_hat_train
    # assume ell_s, ell, sf, sn
    covhyp = np.zeros([4,ytrain.shape[1]]) # for prod of cov kernels
    #covhyp = np.zeros([5,ytrain.shape[1]]) # for sum of cov kernels
    means = np.transpose(linearmodel.coef_)
    for oindex in range(ytrain.shape[1]):
        #check for nzcoefs
        nzcoefs = np.where(means[:,oindex] != 0)[0]
        xtrain1 = xtrain[:,nzcoefs]
        ### need to pass the latlon for train data
        _covhyp = eng.trainGP_spatial_cov(np.transpose(xtrain1).tolist(),
                           np.transpose(noise_train[:,oindex:oindex+1]).tolist(),
                           np.transpose(strain).tolist(),
                           xtrain1.shape[0],xtrain1.shape[1])
        _covhyp = np.array(_covhyp).flatten()
        covhyp[:,oindex] = _covhyp.flatten()
    
    return covhyp

In [ ]:
#predict using GPR
def test_spatial_GP(xtrain,ytrain,strain,xtest,stest,linearmodel,covhyp,eng):
    ystar_test = np.zeros([xtest.shape[0],ytrain.shape[1]])
    ystar_var = np.zeros([xtest.shape[0],ytrain.shape[1]])
    
    means = np.transpose(linearmodel.coef_)
    consts = linearmodel.intercept_
    
    for oindex in range(ytrain.shape[1]):
        #check for nzcoefs
        nzcoefs = np.where(means[:,oindex] != 0)[0]
        
        meanparams = np.zeros([len(nzcoefs)+1,])
        meanparams[0:len(nzcoefs)] = means[nzcoefs,oindex]
        meanparams[len(nzcoefs)] = consts[oindex]
        
        xtrain1 = xtrain[:,nzcoefs]
        xtest1 = xtest[:,nzcoefs]

        _ystar_res = eng.testGP_spatial_cov(np.transpose(xtrain1).tolist(),
                                np.transpose(ytrain[:,oindex:oindex+1]).tolist(),
                                np.transpose(strain).tolist(),
                                np.transpose(xtest1).tolist(),
                                np.transpose(stest).tolist(),
                                xtrain.shape[0],xtest1.shape[0],xtrain1.shape[1],
                                np.transpose(meanparams).tolist(),
                                np.transpose(covhyp[:,oindex:oindex+1]).tolist())
        
        _ystar_res = np.array(_ystar_res).flatten()
        _ystar_var = _ystar_res[xtest1.shape[0]:]
        _ystar_test = _ystar_res[0:xtest1.shape[0]]
        
        ystar_test[:,oindex] = _ystar_test.flatten()
        ystar_var[:,oindex] = _ystar_var.flatten()
        
    return ystar_test, ystar_var 

In [ ]:
#run GP MATLAB model
def run_Mixed(xtrain,ytrain,xtest,eng,alpha):
    linearmodel = ElasticNet(alpha = alpha)
    linearmodel.fit(xtrain,ytrain)
    y_hat_train = linearmodel.predict(xtrain)
    noise_train = ytrain - y_hat_train
    
    
    noise_test = np.zeros([xtest.shape[0],ytrain.shape[1]])
    noise_var = np.zeros([xtest.shape[0],ytrain.shape[1]])
    
    for oindex in range(ytrain.shape[1]):
        _noise_res = eng.runGP_noise(np.transpose(xtrain).tolist(),
                           np.transpose(noise_train[:,oindex:oindex+1]).tolist(),np.transpose(xtest).tolist(),
                           xtrain.shape[0],xtest.shape[0],xtrain.shape[1])
      
        _noise_res = np.array(_noise_res).flatten()
        _noise_var = _noise_res[xtest.shape[0]:]
        _noise_test = _noise_res[0:xtest.shape[0]]
        
        noise_test[:,oindex] = _noise_test.flatten()
        noise_var[:,oindex] = _noise_var.flatten()
    
    y_hat_test = linearmodel.predict(xtest)
    ystar = y_hat_test + noise_test
    
    return ystar,noise_var

In [ ]:
def pretty_print(a):
    st = ''
    for _n in a:
        st = st+'%05.2f '%_n
    print st[:-1]


In [ ]:
def fillMissingCommunes(current_predictions,all_commune_ids):
    # find regional predictions
    reg_predictions = {}
    for c in current_predictions.keys():
        r = c[0:2]
        if r not in reg_predictions.keys():
            reg_predictions[r] = []
        reg_predictions[r].append(current_predictions[c])
    avg_reg_predictions = {}
    for r in reg_predictions.keys():
        avg_reg_predictions[r] = np.mean(np.array(reg_predictions[r]),axis=0)
    new_predictions = {}
    for c in all_commune_ids:
        if c in current_predictions.keys():
            new_predictions[c] = current_predictions[c]
        else:
            new_predictions[c] = avg_reg_predictions[c[0:2]]
    return new_predictions